**Install and Import necessary libraries**


1.   pysam - python wrapper of samtool
2.   samtool - need this to index the bam file
3.   pandas  
4.   numpy
5.   Array 
6.   subprocess - to execute the samtool command in 
     python environment



In [2]:
#uncomment this and run 
#!pip install pysam
#!apt-get install samtools

Here we define a function named **calculate_gene_coverage** which takes two input bam file and gtf file.

# **Step by step execuition of this function**


1.   Index the bam file using 
     samtool via subprocessing.
2.   Read the bam file in read and 
     binary format and align the bam file.
3.   Process the gtf file to get the coordinates of    
     the gene and its name.
4.   Initiate an empty variable named coverage_data so 
     as to append all the output of the algorithms here.
5.   Iterate over all the rows of the gtf file for 
     Gene name , start position and end position.
6.   count_coverage method is used in the bam object. 
     this count_coverage method return 4 array.array 
     list consists of the depth of each nucleotide(ATGC) at each position. like if our ending and starting position will be like this 5 and 2 respectively then the length of each array should be 5 - 2 that is 3 and each 3 position.
     This 3 position will be mapped by the reads 
     and algorithms return the counts of the reads mapped at each position with the respective bases lets suppose. 
     array will be like that( https://pysam.readthedocs.io/en/latest/api.html)
     A([3, 0, 3])
     T([5, 1, 0])
     G([3, 2, 9])
     C([7, 0, 0])

     These are the depth of each base. 

  8. To get the covered percentage we need the covered 
     bases like how many bases are covered.
     as we see in the above example.
     we need those depth which is equals to or greater than 1.
     in the above example we have 8 covered bases.
     so the coverage percentage should be 
      **covbase/(end - start + 1) * 100**

  9. In the end this function return a dataframe which consists of gene name, region name, start and end position, covbases and coverage percentage





In [11]:
#result_df

Mount the google drive to ur colab and locate your respective file here and run the below given code without any changing.

Just need to change here bam_file = "/path/to/your_file.bam" #path to bam file
gtf_file = "/path/to/your_file.gtf" #path to gtf file

In [7]:
import subprocess
import pysam
import pandas as pd

def calculate_gene_coverage(bam_file, gtf_file):
    subprocess.run(["samtools", "index", bam_file])
    bam = pysam.AlignmentFile(bam_file, "rb")
    gene_annotations = pd.read_csv(gtf_file, sep='\t', comment='#', header=None, dtype={0: str})
    gene_annotations = gene_annotations[gene_annotations[2] == "gene"]
    coverage_data = []
    for _, gene in gene_annotations.iterrows():
        chrom = gene[0]
        start = gene[3]
        end = gene[4]
        coverage = bam.count_coverage(chrom, start, end)
        covbases = sum(sum(depth >= 1 for depth in depth_array) for depth_array in coverage)
        covpercent = (covbases / (end - start + 1)) * 100
        gene_name = gene[8].split(";")[0].split(" ")[1].replace('"', '')
        coverage_data.append((gene_name, chrom, start, end, covbases, covpercent))
    df = pd.DataFrame(coverage_data, columns=['Gene', 'Chromosome', 'Start', 'End', 'CovBases', 'CovPercent'])
    bam.close()
    return df

#please specify the correct file location after mounting change the path below

bam_file = "/content/drive/MyDrive/gene wise coverage/IGIB113050041V.bam" #path to bam file
gtf_file = "/content/drive/MyDrive/gene wise coverage/Homo_sapiens.GRCh38.106.gtf" #path to gtf file

result_df_ = calculate_gene_coverage(bam_file, gtf_file)


In [8]:
result_df_

,Gene,Chromosome,Start,End,CovBases,CovPercent
0,ENSG00000186827,1,1211340,1214153,174,6.183369
1,ENSG00000186891,1,1203508,1206592,18,0.583468
2,ENSG00000160072,1,1471765,1497848,4664,17.880693
3,ENSG00000260179,1,1249777,1251334,318,20.410783
4,ENSG00000234396,1,2212523,2220738,843,10.260467
...,...,...,...,...,...,...
61547,ENSG00000271254,KI270711.1,4612,29626,6900,27.583450
61548,ENSG00000275987,KI270713.1,30437,30580,0,0.000000
61549,ENSG00000268674,KI270713.1,35407,35916,154,30.196078
61550,ENSG00000277475,KI270713.1,31698,32528,175,21.058965


In [9]:
#import subprocess
#import pysam
#import pandas as pd
#from array import array

#def calculate_gene_coverage(bam_file, gtf_file):
 #   subprocess.run(["samtools", "index", bam_file])
  #  bam = pysam.AlignmentFile(bam_file, "rb")
   # gene_annotations = pd.read_csv(gtf_file, sep='\t', comment='#', header=None, dtype={0: str})
    #gene_annotations = gene_annotations[gene_annotations[2] == "gene"]
    #coverage_data = []
    #for _, gene in gene_annotations.iterrows():
     #   chrom = gene[0]
      #  start = gene[3]
       # end = gene[4]
       # coverage = bam.count_coverage(chrom,start,end)
        #A = pd.Series(coverage[0])
        #C = pd.Series(coverage[1])
        #G = pd.Series(coverage[2])
        #T = pd.Series(coverage[3])
        #A = pd.DataFrame(A, columns=['A'])
        #T = pd.DataFrame(T, columns=['T'])
        #G = pd.DataFrame(G, columns=['G'])
        #C = pd.DataFrame(C, columns=['C'])
        #A = pd.DataFrame(A) 
        #C = pd.DataFrame(C)
        #G = pd.DataFrame(G)
        #T = pd.DataFrame(T)
        #depthA = A[A['A'] >=1]
        #depthT = T[T['T'] >=1]
        #depthG = G[G['G'] >=1]
        #depthC = C[C['C'] >=1]
        #covbases = depthA.shape[0] + depthT.shape[0] + depthG.shape[0] + depthC.shape[0]
        #covpercent = (covbases / (end - start + 1)) * 100 
        #gene_name = gene[8].split(";")[0].split(" ")[1].replace('"', '')
        #coverage_data.append((gene_name, chrom, start, end, covbases, covpercent))
    #df = pd.DataFrame(coverage_data, columns=['Gene', 'Chromosome', 'Start', 'End', 'CovBases', 'CovPercent'])

    #bam.close()

    #return df

#bam_file = "/content/drive/MyDrive/gene wise coverage/IGIB113050041V.bam"
#gtf_file = "/content/drive/MyDrive/gene wise coverage/Homo_sapiens.GRCh38.106.gtf"

#result_df = calculate_gene_coverage(bam_file, gtf_file)



In [10]:
#result_df.to_csv('IGIB113050041V_gene_wise_coverage.csv')